# Story Time with Markov Twain

Add two workers to do processing.

In [1]:
# addprocs(2);

Remember, Julia interpreter prints last variable of cell.

In [2]:
function clean_corpus(text, regex, normalize = true, lower_case = true)
    if normalize
        # replace control characters with spaces
        text = normalize_string(text, stripmark = true, stripignore = true, stripcc = true)
    end
    
    if lower_case
        text = lowercase(text)
    end
    
    # remove unwanted characters
    text = replace(text, regex, "")
    
    # remove ""
    text = split(text, " ")
    target_index = 1
    for i in 1:length(text)
        target_index = findnext(text, "", target_index)
        if target_index == 0
            break
        else
            splice!(text, target_index)
        end
    end        
    text = join(text, " ")

end;

Read in file

In [3]:
f = open("mark_twain/adventures_of_tom_sawyer.txt")
ats = readall(f);

Clean text

In [4]:
# create regex object (I prefer whitelisting characters I want to keep)
chars_to_remove = r"[^a-z ]"
ats_clean = clean_corpus(ats, chars_to_remove);

Define text to numeric function

In [5]:
function text_to_numeric(symbols, text)
    numeric_text = []
    for each in text
        push!(numeric_text, findfirst(symbols, each))
    end
    # convert to tuple?
    numeric_text
end;

In [37]:
function numeric_to_text(symbols, numeric)
    text= []
    for num in numeric
        push!(text, symbols[num])
    end
    text
end;

In [45]:
function get_corpus_frequencies(corpus, ngram, groupby = "words")
    if groupby == "chars"
        corpus = split(corpus, "")
    else        
        corpus = split(corpus, " ")
    end
    
    # find unique symbols
    unique_symbols = unique(corpus)   
    # convert text to numbers
    corpus_numeric = text_to_numeric(unique_symbols, corpus);
    # create M
    dimensions = repeat([length(unique_symbols)], outer=[ngram])
    M = repeat([0], outer = dimensions)
    # get frequencies for ngram of text
    for i in 1:length(corpus)-ngram+1
        M[corpus_numeric[i:i+ngram-1]...] += 1
    end
    
    M
end;

Let's make sure this frequency array works on a subset of the text.

In [48]:
len_ats_clean = length(split(ats_clean, " "))
# text subset
ats_subset = join(split(ats_clean, " ")[1:round(Int64, len_ats_clean/2)], " ")
@time M = get_corpus_frequencies(ats_subset, 2);

 15.876667 seconds (239.61 M allocations: 7.367 GB, 3.72% gc time)


Now let's take it a step up and get a frequency array for the whole text.

In [49]:
@time M = get_corpus_frequencies(ats_clean, 2);

 36.419307 seconds (529.34 M allocations: 16.274 GB, 3.95% gc time)


This baby purs :)